In [40]:
# To find non-induced inclusion this can be used

def check_containment(smalls, larges):
    """
    Helper function to check is any of the smalls appears in each of the larges.

    INPUT:
    smalls - list of flags, must be from a theory with edges relation
    larges - list of flags, also must be from a theory with edges relation

    OUTPUT:
    list of booleans, i-th element represents if i-th large flag is free from all smalls
    """
    sis = [IncidenceStructure(ss.size(), ss.blocks()['edges']) for ss in smalls]
    lis = [IncidenceStructure(ss.size(), ss.blocks()['edges']) for ss in larges]
    res = []
    for ll in lis:
        good = True
        for ss in sis:
            for _ in ll.isomorphic_substructures_iterator(ss):
                good = False
                break
            if not good:
                break
        res.append(good)
    return res

def check_containment_cert(smalls, large):
    """
    Helper function to check is any of the smalls appears in the large, and if yes,
    returns an injection of the small

    INPUT:
    smalls - list of flags, must be from a theory with edges relation
    large - a flag, also must be from a theory with edges relation

    OUTPUT:
    empty list ([]) if all smalls is avoided, otherwise [small, mapping] telling how to
    inject small into large with mapping
    """
    sis = [IncidenceStructure(ss.size(), ss.blocks()['edges']) for ss in smalls]
    lis = IncidenceStructure(large.size(), large.blocks()['edges'])
    for ii, ss in enumerate(sis):
        for xx in lis.isomorphic_substructures_iterator(ss):
            return [xx, smalls[ii]]
    return None

In [41]:
from sage.algebras.flag_algebras import *

# This is some hack to create the theory for 3-graphs without C5- and K4-
# up to size 7. It is easier to make them as extensions of 6 sized structures
# so this code does that.

# Reset three graphs, so nothing is excluded
TG = ThreeGraphTheory
# k4 and k4m (the two induced structures with size 4 excluded)
k4 = TG(4, edges=[[0, 1, 2], [0, 1, 3], [0, 2, 3], [1, 2, 3]])
k4m = TG(4, edges=[[0, 1, 2], [0, 1, 3], [0, 2, 3]])
TG.exclude([k4,k4m])

# check the list of flags with size 5 and 6
fl5 = TG.generate_flags(5)
fl6 = TG.generate_flags(6)

# quick 3-graph identifier code. This will be the identifier for
# the theory of C5- free 3-graphs (any identifier working for 3-graphs can work here)
def _identify_hypergraph(n, ftype_points, edges):
    g = Graph([list(range(n+len(edges))), [(i+n,x) for i,b in enumerate(edges) for x in b]], 
              format='vertices_and_edges')
    partt = [[ii] for ii in ftype_points] + \
            [[ii for ii in range(n) if ii not in ftype_points]] + \
            [list(range(n,n+len(edges)))]
    blocks = tuple(g.canonical_label(partition=partt).edges(labels=None, sort=True))
    ftype_points = tuple(range(len(ftype_points)))
    return (n, ftype_points, blocks)

# generator code. It should really just return TG, but for size 7 that takes too long
# so this hack just returns TG for size up to 6, and for 7 it generates all flags
# with this extension technique
def _gen(n):
    if n<=4:
        for xx in TG.generate_flags(n):
            yield xx.blocks()
    elif n==5:
        for xx in fl5:
            yield xx.blocks()
    elif n==6:
        for xx in fl6:
            yield xx.blocks()
    elif n==7:
        import itertools
        from tqdm import tqdm
        fl7_m = [[] for ii in range(35+1)]
        subs = list(itertools.combinations(range(6), int(2)))
        for xx in tqdm(fl6):
            xb = xx.blocks()['edges']
            for ii in range(15+1):
                for pps in itertools.combinations(subs, int(ii)):
                    xbp = [[pp[0], pp[1], 6] for pp in pps] + xb
                    flxp = TG(7, edges=xbp)
                    en = len(xbp)
                    if flxp not in fl7_m[en]:
                        if check_containment(exls, [flxp])[0]:
                            fl7_m[en].append(flxp)
        fl7 = [yy for xx in fl7_m for yy in xx]
        for xx in fl7:
            yield xx.blocks()
    else:
        #for n>=8 just return an empty list, this will not be called so doesn't 
        #really matter
        return []

# Create the theory based on this generator and identifier
TGp = CombinatorialTheory("NoK4m", _gen, _identify_hypergraph, edges=3)

# for sanity check, print the number of structures with size 5, 6, 7
# should be 9 55 1127
print(len(TGp.generate_flags(5)), len(TGp.generate_flags(6)), len(TGp.generate_flags(7)))

11 106 8157


In [42]:
from itertools import combinations
def get_supergraphs(base):
    missing_edges = []
    for e in combinations(range(6), 3):
        f = [v for v in e]
        if f not in base:
            missing_edges.append(f)
            
    graphs = []
    for i in range(0, len(missing_edges)):
        for E in combinations(missing_edges, i):
            M = base + [e for e in E]
            k4_ = False
            for t in combinations(range(6), 4):
                numb_edges = 0
                for e in M:
                    counts = True
                    for v in e:
                        if v not in t:
                            counts = False
                            break
                    if counts:
                        numb_edges += 1
                if numb_edges >= 3: # Contains K_4^-
                    k4_ = True
                    break
            if not k4_:
                graphs.append(M)
    return graphs

In [43]:
base_graph = [[0, 1, 2], [3, 4, 5], [1, 2, 3], [1, 2, 4], [1, 2, 5]]
additional_graph = [[0, 1, 2], [3, 4, 5], [1, 2, 3], [1, 2, 4], [1, 2, 5], [0, 3, 4]]

f411 = TGp(6, ftype = [0, 1, 2], edges = base_graph) + TGp(6, ftype = [0, 1, 2], edges = additional_graph)
    
pointed_edge = TGp(3, edges=[[0, 1, 2]], ftype=[0])

p2f4 = TGp.generate_flags(4, TGp(2, ftype=[0, 1]))
degree_difference = p2f4[2]-p2f4[3]+p2f4[5]-p2f4[6]

assump = [pointed_edge-2/7, degree_difference, -degree_difference]

min_f411 = TGp.optimize_problem(f411, 6, maximize=True, positives=assump)

print("\n\n", min_f411)

Ftypes constructed in 0.24s
Block sizes done in 0.00s
Block sizes are [8, 41, 26, 18, -106, -46]
Calculating product matrices for 4 ftypes and 106 structures
Ftype on 4 points with edges=[[0, 1, 2], [0, 1, 3]] is complete: : 4it [00:00, 152.10it/s]
Table calculation done in 0.03s
Target and constraint calculation done in 0.04s

CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 8.64e-01 Pobj: -6.3190279e+01 Ad: 3.12e-01 Dobj: -1.9944135e-02 
Iter:  2 Ap: 1.00e+00 Pobj: -9.2333974e+01 Ad: 7.20e-01 Dobj: -8.6750881e-03 
Iter:  3 Ap: 1.00e+00 Pobj: -9.8620208e+01 Ad: 9.07e-01 Dobj: -3.1689590e-03 
Iter:  4 Ap: 1.00e+00 Pobj: -1.0457944e+02 Ad: 6.93e-01 Dobj: -2.7250362e-03 
Iter:  5 Ap: 5.32e-01 Pobj: -7.4342411e+01 Ad: 4.73e-01 Dobj: -3.2294849e-03 
Iter:  6 Ap: 6.02e-01 Pobj: -6.0611096e+01 Ad: 6.84e-01 Dobj: -3.5634038e-03 
Iter:  7 Ap: 9.77e-01 Pobj: -4.2281776e+01 Ad: 6.77e-01 Dobj: -2.7477718e-03 
Iter:  8 Ap: 9.05e-01 Pobj: -2.4866

In [44]:
base = [[0, 1, 2], [3, 4, 5], [1, 2, 3], [1, 2, 4], [0, 2, 5]]
uniques = []
for eds in get_supergraphs(base):
    fl = TGp(6, edges=eds, ftype=[0, 1, 2])
    if fl not in uniques:
        uniques.append(fl)

f321 = -1
for xx in uniques:
    f321 += xx
f321 += 1

max_f321 = TGp.optimize_problem(f321, 6, maximize=False, positives=assump)

print("\n\n", max_f321)

Ftypes constructed in 0.23s
Block sizes done in 0.00s
Block sizes are [8, 41, 26, 18, -106, -46]
Calculating product matrices for 4 ftypes and 106 structures
Ftype on 4 points with edges=[[0, 1, 2], [0, 1, 3]] is complete: : 4it [00:00, 158.26it/s]
Table calculation done in 0.03s
Target and constraint calculation done in 0.13s

CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 8.63e-01 Pobj: -6.3264706e+01 Ad: 3.11e-01 Dobj: -6.6268927e-02 
Iter:  2 Ap: 1.00e+00 Pobj: -9.2592353e+01 Ad: 7.19e-01 Dobj: -2.8586375e-02 
Iter:  3 Ap: 1.00e+00 Pobj: -9.8920434e+01 Ad: 9.07e-01 Dobj:  2.2956117e-03 
Iter:  4 Ap: 1.00e+00 Pobj: -1.0495399e+02 Ad: 6.94e-01 Dobj:  4.7795342e-03 
Iter:  5 Ap: 5.38e-01 Pobj: -7.4507885e+01 Ad: 4.75e-01 Dobj:  7.0342354e-03 
Iter:  6 Ap: 6.01e-01 Pobj: -6.0829547e+01 Ad: 6.84e-01 Dobj:  1.2526429e-02 
Iter:  7 Ap: 9.78e-01 Pobj: -4.2335072e+01 Ad: 6.77e-01 Dobj:  2.4568832e-02 
Iter:  8 Ap: 9.09e-01 Pobj: -2.4785

In [45]:
base_graph = [[0, 1, 2], [3, 4, 5], [1, 2, 3], [0, 2, 4], [0, 1, 5]]
uniques = []
for eds in get_supergraphs(base):
    fl = TGp(6, edges=eds, ftype=[0, 1, 2])
    if fl not in uniques:
        uniques.append(fl)

f222 = -1
for xx in uniques:
    f222 += xx
f222 += 1

max_f222 = TGp.optimize_problem(f222, 6, maximize=False, positives=assump)

print("\n\n", max_f222)

Ftypes constructed in 0.24s
Block sizes done in 0.00s
Block sizes are [8, 41, 26, 18, -106, -46]
Calculating product matrices for 4 ftypes and 106 structures
Ftype on 4 points with edges=[[0, 1, 2], [0, 1, 3]] is complete: : 4it [00:00, 156.87it/s]
Table calculation done in 0.03s
Target and constraint calculation done in 0.14s

CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 8.63e-01 Pobj: -6.3264706e+01 Ad: 3.11e-01 Dobj: -6.6268927e-02 
Iter:  2 Ap: 1.00e+00 Pobj: -9.2592353e+01 Ad: 7.19e-01 Dobj: -2.8586375e-02 
Iter:  3 Ap: 1.00e+00 Pobj: -9.8920434e+01 Ad: 9.07e-01 Dobj:  2.2956117e-03 
Iter:  4 Ap: 1.00e+00 Pobj: -1.0495399e+02 Ad: 6.94e-01 Dobj:  4.7795342e-03 
Iter:  5 Ap: 5.38e-01 Pobj: -7.4507885e+01 Ad: 4.75e-01 Dobj:  7.0342354e-03 
Iter:  6 Ap: 6.01e-01 Pobj: -6.0829547e+01 Ad: 6.84e-01 Dobj:  1.2526429e-02 
Iter:  7 Ap: 9.78e-01 Pobj: -4.2335072e+01 Ad: 6.77e-01 Dobj:  2.4568832e-02 
Iter:  8 Ap: 9.09e-01 Pobj: -2.4785

In [46]:
max_f222 = TGp.optimize_problem((f321 + f222)/2, 6, maximize=False, positives=assump)

print("\n\n", max_f222)

Ftypes constructed in 0.25s
Block sizes done in 0.01s
Block sizes are [8, 41, 26, 18, -106, -46]
Calculating product matrices for 4 ftypes and 106 structures
Ftype on 4 points with edges=[[0, 1, 2], [0, 1, 3]] is complete: : 4it [00:00, 172.68it/s]
Table calculation done in 0.02s
Target and constraint calculation done in 0.04s

CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Result is 0.022310925068318627
Iter:  1 Ap: 8.63e-01 Pobj: -6.3264706e+01 Ad: 3.11e-01 Dobj: -6.6268927e-02 
Iter:  2 Ap: 1.00e+00 Pobj: -9.2592353e+01 Ad: 7.19e-01 Dobj: -2.8586375e-02 
Iter:  3 Ap: 1.00e+00 Pobj: -9.8920434e+01 Ad: 9.07e-01 Dobj:  2.2956117e-03 
Iter:  4 Ap: 1.00e+00 Pobj: -1.0495399e+02 Ad: 6.94e-01 Dobj:  4.7795342e-03 
Iter:  5 Ap: 5.38e-01 Pobj: -7.4507885e+01 Ad: 4.75e-01 Dobj:  7.0342354e-03 
Iter:  6 Ap: 6.01e-01 Pobj: -6.0829547e+01 Ad: 6.84e-01 Dobj:  1.2526429e-02 
Iter:  7 Ap: 9.78e-01 Pobj: -4.2335072e+01 Ad: 6.77e-01 Dobj:  2.4568832e-02 
Iter